In [2]:
import netCDF4
import numpy as np

input_ncfile_path = 'D:\VSCODE\weather_resource\gk2a_ir096_2021\gk2a_ami_le1b_ir096_ko020lc_202101010000.nc' 
CT_path='./conversion_table/'
output_txt_path='sample_output.txt'
output_npy_path='sample_image_pixel_data.npy'
ncfile = netCDF4.Dataset(input_ncfile_path,'r',format='netcdf4')
ipixel=ncfile.variables['image_pixel_values']
ipixel_process = ipixel[:]


In [3]:
channel=ipixel.getncattr('channel_name')
if ((channel == 'VI004') or (channel == 'NR016')
    or (channel == 'VI005')):
        mask = 0b0000011111111111 #11bit mask
elif ((channel == 'VI006')
    or (channel == 'NR013') or (channel == 'WV063')):
        mask = 0b0000111111111111 #12bit mask
elif (channel == 'SW038'):
    mask = 0b0011111111111111 #14bit mask
else:
    mask = 0b0001111111111111 #13bit mask
    ipixel_process_masked=np.bitwise_and(ipixel_process,mask)

In [4]:
channel

'IR096'

In [26]:
AL_postfix='_con_alb.txt'
BT_postfix='_con_bt.txt'
if (channel[0:2] == 'VI') or (channel[0:2] == 'NR'):
    conversion_table=np.loadtxt(
    CT_path+channel+AL_postfix,'float64')
else:
    conversion_table=np.loadtxt(
        CT_path+channel+BT_postfix,'float64')
ipixel_process_masked_converted=conversion_table[
    ipixel_process_masked]

In [6]:
ipixel.shape

(900, 900)

In [7]:
ipixel_process.shape

(900, 900)

In [8]:
ipixel_process[0,0]

6977

In [9]:
conversion_table[6977]

231.5546

In [10]:
ipixel_process

masked_array(
  data=[[6977, 6968, 6954, ..., 6655, 6654, 6653],
        [6977, 6968, 6951, ..., 6654, 6654, 6656],
        [6981, 6971, 6970, ..., 6654, 6654, 6656],
        ...,
        [6220, 6208, 6202, ..., 5425, 5410, 5436],
        [6197, 6208, 6202, ..., 5472, 5481, 5436],
        [6197, 6198, 6223, ..., 5398, 5425, 5453]],
  mask=False,
  fill_value=999999,
  dtype=uint16)

In [23]:
ipixel_process_masked_converted

array([[231.5546, 231.8281, 232.2505, ..., 240.5578, 240.5836, 240.6094],
       [231.5546, 231.8281, 232.3406, ..., 240.5836, 240.5836, 240.532 ],
       [231.4327, 231.7371, 231.7674, ..., 240.5836, 240.5836, 240.532 ],
       ...,
       [250.8244, 251.0844, 251.2141, ..., 266.1591, 266.4182, 265.9685],
       [251.3219, 251.0844, 251.2141, ..., 265.3412, 265.1835, 265.9685],
       [251.3219, 251.3003, 250.7592, ..., 266.6248, 266.1591, 265.6729]])

In [27]:
for i in range(900):
    for j in range(900):
        ipixel_process_masked_converted[i,j]=8191-ipixel_process_masked_converted[i,j]
        ipixel_process_masked_converted[i,j]/=8191
        ipixel_process_masked_converted[i,j]*=255

In [28]:
ipixel_process_masked_converted

array([[247.79130472, 247.7827902 , 247.76964015, ..., 247.51101953,
        247.51021634, 247.50941314],
       [247.79130472, 247.7827902 , 247.76683518, ..., 247.51021634,
        247.51021634, 247.51182273],
       [247.79509968, 247.78562318, 247.78467989, ..., 247.51021634,
        247.51021634, 247.51182273],
       ...,
       [247.19140251, 247.18330827, 247.17927048, ..., 246.71400678,
        246.70594054, 246.71994048],
       [247.17591448, 247.18330827, 247.17927048, ..., 246.73946942,
        246.74437889, 246.71994048],
       [247.17591448, 247.17658692, 247.1934323 , ..., 246.69950873,
        246.71400678, 246.72914302]])

In [29]:
import imageio

imageio.imsave("ddd.png", ipixel_process_masked_converted)

Lossy conversion from float64 to uint8. Range [246.6845717250641, 247.90937492369676]. Convert image to uint8 prior to saving to suppress this warning.
